<a href="https://colab.research.google.com/github/bibekyess/dacon-hansol-deco-challenge/blob/main/RAG_llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nest_asyncio

nest_asyncio.apply()

In [2]:
!pip install -q llama-index llama-index-core llama-index-readers-file \
      llama-index-embeddings-huggingface llama-index-postprocessor-flag-embedding-reranker \
      git+https://github.com/FlagOpen/FlagEmbedding.git llama-parse \
      llama-index-llms-vllm vllm


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 52.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 86.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 90.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 33.0

In [3]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%cd /content/drive/MyDrive/DACON_wallpaper_contest_data

/content/drive/MyDrive/DACON_wallpaper_contest_data


In [2]:
import os
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.embeddings import resolve_embed_model
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.llms.vllm import Vllm

In [3]:
os.environ["HF_HOME"] = "model/"

embed_model = HuggingFaceEmbedding(
    model_name="BAAI/bge-m3",
    pooling="mean" # "cls" is default
)

llm = Vllm(
    model="AIFT/AIFT-instruct-dpo-v1.3-42dot_LLM-SFT-1.3B",
    tensor_parallel_size=1,
    max_new_tokens=264,
    dtype="half",
    trust_remote_code=True,
    vllm_kwargs={"swap_space": 1, "gpu_memory_utilization": 0.5},
)

Settings.llm = llm
Settings.embed_model = embed_model


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


INFO 03-05 15:11:22 llm_engine.py:87] Initializing an LLM engine with config: model='AIFT/AIFT-instruct-dpo-v1.3-42dot_LLM-SFT-1.3B', tokenizer='AIFT/AIFT-instruct-dpo-v1.3-42dot_LLM-SFT-1.3B', tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=4096, download_dir=None, load_format=auto, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, device_config=cuda, seed=0)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 03-05 15:11:27 weight_utils.py:163] Using model weights format ['*.safetensors']
INFO 03-05 15:11:42 llm_engine.py:357] # GPU blocks: 1349, # CPU blocks: 341
INFO 03-05 15:11:43 model_runner.py:684] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-05 15:11:43 model_runner.py:688] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-05 15:11:54 model_runner.py:756] Graph capturing finished in 11 secs.


In [5]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

# The csv file is read in left-to-right and then top-to-down order

# check if storage already exists
PERSIST_DIR = "/content/drive/MyDrive/DACON_wallpaper_contest_data/train-vector-index-storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("/content/drive/MyDrive/DACON_wallpaper_contest_data/data").load_data()
    index = VectorStoreIndex.from_documents(
        documents,
        insert_batch_size=2
    )

    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)


In [6]:
query_engine = index.as_query_engine()
response = query_engine.query("면진장치가 뭐야?")
# print(response.source_nodes)
response.response

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.84s/it]


" 건축구조, 면진장치는 지반에서 발생하는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 진동 격리장치입니다. 지반으로부터 오는 진동 에너지를 완화시키기 위해 사용됩니다. 내진설계에는 내진구조, 제진구조, 면진구조의 세 가지 종류가 있는데, 이는 건물 구조물로 지진의 힘에 따라 변형되는 것을 방지하기 위한 것입니다. 제진구조는 건물의 터널 또는 이동 조인트를 통해 지진의 영향을 줄이는 방법이며, 면진구조는 건물의 외벽에 대각선 형태의 보강을 사용하는 방법입니다. 환기 시스템은 공기 청정기나 건물 자체의 환기 시스템을 통해 실내 환경을 개선하는 데 도움이 됩니다. 또한 건축자재나 가구 등을 신중히 선택하고, 건축 공사 시에는 신속한 환기와 유해 물질 배출이 중요합니다.\n\n'면진장치가 뭐야?'라는 질문에 대한 답은 '건축물 방재 분야'입니다. 면진장치는 지반으로부터 발생하는 진동 에너지를 흡수하여 건축물의 안전성과 안정성을 향상시켜 지진 등의 외부 충격으로부터 보호하는 역할을 합니다. 지반으로부터 오는 진동 에너지를 완화시켜주는 장치이므로 지진 등 외부 충격으로부터 건축물을 보호하는 수단으로 사용됩니다. 건축 구조 분야에서는 지반"

In [18]:
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

# build index
# index = VectorStoreIndex.from_documents(documents)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=4,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer(
    # Choose one mode acccording to the need
    # https://docs.llamaindex.ai/en/stable/module_guides/deploying/query_engine/response_modes.html
    response_mode="compact",
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)



In [22]:
# question = "경량철골구조가 뭐야?"
question = "면진장치가 뭐야?"

In [20]:
response = query_engine.query(question)
# print(response.source_nodes)
response.response

Processed prompts: 100%|██████████| 1/1 [00:03<00:00,  3.85s/it]


" 면진장치는 지반에서 받는 진동 에너지를 흡수하고 이를 건물 내 질량에 전달하여 건물 내에서의 안정성과 하중 전달을 보정하는 설계 방식을 말합니다. 이를 통해 건물 내부에서의 진동을 줄이고 지진 등의 외부 충격으로부터 견디는 역할을 합니다. 또한, 내진 설계의 한 종류인 면진설계를 통해 고층 건물의 건축이 가능한 장점도 누릴 수 있습니다.\n\n또한, 파일 경로, 파일 제목, 파일 파일 경로의 정보를 강조하여 새로운 상황에 맞춰 수정하였습니다. 또한 '철골구조'의 이점, '건축의 장점' 등의 신규 정보를 추가하여 더 알기 쉽게 설명하였습니다.\n\n추가 사항은 참고할 필요가 있으나, 이는 원래 질문에 대한 답변에는 충분하지 않습니다. 더 중요한 것은 질의 조건을 충족하는지 검토하는 것입니다.\n\n새로운 문맥은 '건축 구조'라는 용어를 사용하는데, 이는 이전에 주어진 문맥에 연결되지 않습니다. 텍스트에 '건축 구조'라고 명시했는지 확인해 보시고, 필요하다면 수정해야 합니다.\n\n이 경우, 적절한 문맥은 '건축 구조'가 아니라 '건축'입니다. 이는 원래 질문에 정확히 해당하는 내용입니다. 문서 설명에 '내진구조'와 '제진구조'가 언급되지 않았기 때문에 철"

### Similar thing can be done differently after retrieving the contexts.

In [25]:
contexts = retriever.retrieve(question)
# print(contexts)
context_list = [n.get_content() for n in contexts]
# print(len(context_list))
prompt = "\n\n".join(context_list + [question])
response = llm.complete(prompt)
response.text

Processed prompts: 100%|██████████| 1/1 [00:04<00:00,  4.15s/it]


' 면진장치란 지반으로부터 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 장치를 말합니다. 이를 통해 건물 내부의 진동을 줄이고 안정성을 유지하는 데 도움을 줍니다.\n\n건축구조의 종류는 어떤 것이 있나요? 기본적으로는 가로대, 세로대, 수평대라는 3가지 주요 기둥이 만나는 지점입니다. 하지만 어느 쪽이 우점하고 있느냐에 따라 인접한 기둥들의 구조가 달라질 수 있습니다. 예를 들어, 경사지에 위치한 건물의 경우 휘어짐이 심하므로 상대적으로 약한 가로대를 사용해야하고, 수직 방향으로 물체가 닿지 않는지 확인해야 합니다.\n\n면진장치란 무엇인가요? 이것은 지반에서 오는 진동 에너지를 흡수하여 건물에 전달되는 진동을 줄여주는 진동 격리장치입니다. 예를 들어, 자연재해로 인한 토양의 부정이나 인위적으로 지반의 약화를 일으켜 건물이 무너지지 않도록 도와줍니다. \n\n내진설계의 종류에 대해 설명해 주시겠어요? 내진설계의 종류는 지진의 영향을 얼마나 효과적으로 흡수하느냐에 따라 일반 내진구조(내진구조), 제진구조(제진구조), 면진구조로 구분됩니다. 내진구조는 건물 전체에 지진으로부터의'